In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("data.csv")

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
df.columns=df.columns.str.lower().str.replace(" ","_")
strings=list(df.dtypes[df.dtypes=="object"].index)
for col in strings:
    df[col]=df[col].str.lower().str.replace(" ","_")
n=len(df)
n_val=int(n*0.2)
n_test=int(n*0.2)
n_train=n-n_test-n_val
ids=np.arange(n)
np.random.shuffle(ids)
df_train=df.iloc[ids[:n_train]].reset_index(drop=True)
df_val=df.iloc[ids[n_train:n_train+n_val]].reset_index(drop=True)
df_test=df.iloc[ids[n_train+n_val:]].reset_index(drop=True)

In [4]:
y_train=np.log1p(df_train.msrp.values)

In [5]:
y_test=np.log1p(df_test.msrp.values)

In [6]:
y_val=np.log1p(df_val.msrp.values)

In [7]:
del df_train["msrp"]
del df_val["msrp"]
del df_test["msrp"]

In [8]:
w0=7.17
w=[0.01,0.04,0.002]

In [9]:

w0=7.17
w=[0.01,0.04,0.002]


In [10]:
base=["engine_hp","engine_cylinders","highway_mpg","city_mpg","popularity"]
c_train=df_train[base].values

In [11]:
x_train=df_train[base].fillna(0).values

In [12]:
def rmse(y,y_pred):
    error=y-y_pred
    se=error**2
    mse=se.mean()
    return np.sqrt(mse)

In [13]:
categoricals=list(df.dtypes[(df.dtypes==object)].index)

In [14]:
categories={}
for c in categoricals:
    categories[c]=list(df[c].value_counts().head().index)

In [15]:
def prepare_X(df):
    df=df.copy()
    features=base+["age"]
    df["age"]=2017-df.year
    features.append("age")
    
    for v in [2,3,4]:
        df["num_doors_%s" % v]=(df.number_of_doors==v).astype("int")
        features.append("num_doors_%s" % v)
    for c, values in categories.items():
        for v in values:
            df["%s_%s" % (c,v)]=(df[c]==v).astype("int")
            features.append("%s_%s" % (c,v))
    df_num=df[features]
    df_num=df_num.fillna(0)
    X=df_num.values
    return X

In [16]:
def train_linear_regression_reg(X,y,r=0.001):
    ones=np.ones(X.shape[0])
    X=np.column_stack([ones,X])
    XTX=X.T.dot(X)
    XTX+=r*np.eye(XTX.shape[0])
    XTX_inv=np.linalg.inv(XTX)
    w_full=XTX_inv.dot(X.T).dot(y)
    return w_full[0],w_full[1:]

In [17]:
r=0.001
X_train=prepare_X(df_train)
w0,w=train_linear_regression_reg(X_train,y_train,r)
X_val=prepare_X(df_val)
y_pred=w0+X_val.dot(w)
rmse(y_val,y_pred)

0.44373588255026997

In [41]:
df_full_train=pd.concat([df_train,df_val]).reset_index(drop=True)

In [42]:
X_full_train=prepare_X(df_full_train)

In [43]:
y_train_full=np.concatenate([y_train,y_val])

In [44]:
w0,w=train_linear_regression_reg(X_full_train,y_train_full,r=0.001)

In [45]:
X_test=prepare_X(df_test)
y_pred=w0+X_test.dot(w)
rmse(y_test,y_pred)

0.45499816834199264

In [49]:
def predict(params):
    y_pred=[]
    for a in range(len(params)):
        x=pd.DataFrame([params.iloc[a].to_dict()])
        X=prepare_X(x)
        y_pred.append(np.expm1((w0+X.dot(w))[0]))
        
    return y_pred